# 智能旅行助手 - ReAct 智能体演示 (基于 Core-Logic 分离架构)

本 Notebook 演示如何使用新的工程架构（Core 基建 + Logic 业务）来运行智能旅行助手。

## 1. 架构说明
- **Core**: 复用了通用的 `llm_client` 和 `ui_utils`。
- **Logic**: 业务逻辑（Prompt 和工具）封装在 `agent_logic.py` 中。
- **Demo**: 本 Notebook 仅负责流程编排和展示。

In [ ]:
# 安装必要的依赖包
!pip install requests tavily-python openai python-dotenv pandas matplotlib

## 2. 定义 ReAct 循环

这里我们定义一个通用的 ReAct 循环函数，它使用 `llm_client` 来调用模型，并使用 `agent_logic` 中的工具。

In [1]:
# === 1. 环境初始化 ===
import sys
import os
from pathlib import Path

# 将当前目录添加到 sys.path，以便导入 core 和 agent_logic
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))

# 导入 Core 基建
from core import get_client_for_model, print_html, check_api_keys

# 导入业务逻辑
from agent_logic import AGENT_SYSTEM_PROMPT, TOOLS

print("✅ 环境初始化完成")
check_api_keys()

✅ 环境初始化完成
🔑 API Keys Configuration Status:
  ❌ OpenAI
  ❌ Anthropic
  ✅ Qwen
  ✅ Zhipu
  ✅ DeepSeek
  ✅ Moonshot



## 3. 运行演示

现在我们可以运行智能体了。请确保 `.env` 文件中配置了正确的 API Key。

In [2]:
import re
import json

def run_react_agent(user_query: str, model_name: str = "glm-4-flash", max_steps: int = 5):
    """
    执行 ReAct 循环
    """
    # 初始化对话历史
    messages = [
        {"role": "system", "content": AGENT_SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]
    
    print_html(f"🚀 开始任务: {user_query}", title="System Start")
    
    for step in range(max_steps):
        # 1. 调用 LLM 获取思考和行动
        client = get_client_for_model(model_name)
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.1
        )
        content = response.choices[0].message.content
        
        # 展示思考过程
        print_html(content, title=f"Step {step + 1}: Thought & Action")
        messages.append({"role": "assistant", "content": content})
        
        # 2. 解析 Action
        # 匹配 Action: function_name(arg="val")
        action_match = re.search(r"Action:\s*(\w+)\((.*)\)", content)
        
        if not action_match:
            # 如果没有 Action，可能是直接回答或格式错误
            if "finish" in content:
                break
            continue
            
        func_name = action_match.group(1)
        args_str = action_match.group(2)
        
        # 3. 执行工具
        if func_name == "finish":
            # 提取最终答案
            answer_match = re.search(r'answer="(.*)"', args_str)
            final_answer = answer_match.group(1) if answer_match else args_str
            print_html(final_answer, title="✅ Final Answer")
            return final_answer
            
        elif func_name in TOOLS:
            try:
                # 简单的参数解析 (生产环境建议用更健壮的解析)
                # 这里假设参数格式比较简单，直接用 eval (仅用于演示，生产环境请勿使用 eval)
                # 为了安全，我们手动解析一下 key=value
                kwargs = {}
                # 简单的正则提取参数
                for arg in args_str.split(','):
                    if '=' in arg:
                        k, v = arg.split('=', 1)
                        kwargs[k.strip()] = v.strip().strip('"').strip("'")
                
                tool_func = TOOLS[func_name]
                observation = tool_func(**kwargs)
                
            except Exception as e:
                observation = f"Error executing {func_name}: {str(e)}"
                
            print_html(observation, title=f"👁️ Observation ({func_name})")
            
            # 4. 将观察结果反馈给 LLM
            messages.append({"role": "user", "content": f"Observation: {observation}"})
            
        else:
            error_msg = f"Error: Tool '{func_name}' not found."
            print_html(error_msg, title="❌ Error")
            messages.append({"role": "user", "content": f"Observation: {error_msg}"})
            
    return "Max steps reached without final answer."

In [3]:
# 示例任务
USER_QUERY = "请帮我查一下北京现在的天气，并推荐一个适合这个天气的旅游景点。"

# 运行 (请确保 .env 中配置了 ZHIPU_API_KEY 和 TAVILY_API_KEY)
# 如果没有 Key，这里会报错
try:
    run_react_agent(USER_QUERY)
except Exception as e:
    print_html(f"运行失败: {str(e)}\n请检查 .env 文件配置。", title="❌ Error")